
## Maximum Likelihood

**Functions**

`np.log`, `scipy.special.gamma`, `scipy.special.gammaln`, `scipy.stats.norm.cdf`, `scipy.optimize.minimize`,
`scipy.stats.t`, `np.var`, `np.std`, `scipy.stats.norm.pdf`

### Exercise 19

Simulate a set of i.i.d. Student's t random variables with degree of freedom
parameter $\nu=10$. Standardize the residuals so that they have unit variance
using the fact that $V\left[x \right]=\frac{\nu}{\nu-2}$. Use these to estimate
the degree of freedom using maximum likelihood. Note that the likelihood of
a standardized Student's t is

$$f(x;\nu,\mu,\sigma^{2})=\frac{\Gamma\left(\frac{\nu+1}{2}\right)}{\Gamma\left(\frac{\nu}{2}\right)}\,\frac{1}{\sqrt{\pi(\nu-2)}}\,\frac{1}{\sigma}\,\frac{1}{\left(1+\frac{\left(x-\mu\right)^{2}}{\sigma^{2}(\nu-2)}\right)^{\frac{\nu+1}{2}}}$$

where $\Gamma\left(\right)$ is known as the gamma function.

In [1]:
import numpy as np
rs = np.random.RandomState(19991231)
var = 10 / (10 - 2)
rvs = rs.standard_t(10, size=1000) / np.sqrt(var)
print(f"The variance is {rvs.var()}")

The variance is 1.0435286272711175


In [2]:
from scipy.special import gammaln

def std_t_loglik(nu, x):
    # These are fixed for now
    mu = 0
    sigma2 = 1
    sigma = np.sqrt(sigma2)
    
   
    a = gammaln((nu + 1) / 2)
    b = gammaln(nu / 2)
    c = np.sqrt(np.pi * (nu-2))
    d = ((nu + 1) / 2)
    e = (x - mu) **2
    f = sigma2 * (nu - 2)
    
    loglik = a - b - np.log(c) - np.log(sigma) - d * np.log(1 + e / f)
    return -(loglik.sum())

print(f"The log like at the population nu is {std_t_loglik(10, rvs)}")
                

The log like at the population nu is 1435.8970961055188


In [3]:
from scipy.optimize import minimize

starting_val = np.array([10])
opt = minimize(std_t_loglik, starting_val, args=(rvs,), bounds=[(2.05, 100)],
               options={"disp": True})
print(f"The MLE is {opt.x[0]} and the optimized (negative) LLF is {opt.fun}")


The MLE is 12.65172718249684 and the optimized (negative) LLF is 1435.6002951852095


### Exercise 20

Repeat the previous exercise using daily, weekly and monthly S&P 500 and Hang Seng data.
Note that it is necessary to remove the mean and standardize by the standard deviation
error before estimating the degree of freedom. What happens over longer horizons?

In [4]:
import pandas as pd

prices = pd.read_hdf("data/equity-indices.h5", "sp500")
rets = 100 * prices.Close.pct_change().dropna()

std_rets = (rets - rets.mean()) / rets.std()
starting_val = np.array([10])
opt = minimize(std_t_loglik, starting_val, args=(std_rets,), bounds=[(2.05, 100)],
               options={"disp": True})
print(f"The MLE is {opt.x[0]} and the optimized (negative) LLF is {opt.fun}")



The MLE is 3.3436180016993595 and the optimized (negative) LLF is 22731.853179026202


In [5]:
keys = ["sp500", "weekly_sp500", "monthly_sp500", "hsi", "weekly_hsi", "monthly_hsi"]
for key in keys:
    prices = pd.read_hdf("data/equity-indices.h5", key)
    rets = 100 * prices.Close.pct_change().dropna()
    
    std_rets = (rets - rets.mean()) / rets.std()
    starting_val = np.array([10])
    opt = minimize(std_t_loglik, starting_val, args=(std_rets,), bounds=[(2.05, 100)],
                   options={"disp": True})
    print(f"For {key}, the MLE is {opt.x[0]} and the optimized (negative) LLF is {opt.fun}")
    


For sp500, the MLE is 3.3436180016993595 and the optimized (negative) LLF is 22731.853179026202
For weekly_sp500, the MLE is 4.877079615300741 and the optimized (negative) LLF is 4948.088538563845


For monthly_sp500, the MLE is 7.195886679997398 and the optimized (negative) LLF is 1169.4687637821962


For hsi, the MLE is 3.3436180016993595 and the optimized (negative) LLF is 22731.853179026202


For weekly_hsi, the MLE is 3.8770233518024906 and the optimized (negative) LLF is 2270.5381886061805
For monthly_hsi, the MLE is 4.2998676523610735 and the optimized (negative) LLF is 531.4943348983934


### Exercise 21

Repeat the previous problem by estimating the mean and variance simultaneously with
the degree of freedom parameter.

In [6]:
def full_std_t_loglik(parameters, x):
    # These are fixed for now
    mu = parameters[0]
    sigma2 = parameters[1]
    nu = parameters[2]
    sigma = np.sqrt(sigma2)
   
    a = gammaln((nu + 1) / 2)
    b = gammaln(nu / 2)
    c = np.sqrt(np.pi * (nu-2))
    d = ((nu + 1) / 2)
    e = (x - mu) **2
    f = sigma2 * (nu - 2)
    
    loglik = a - b - np.log(c) - np.log(sigma) - d * np.log(1 + e / f)
    return -(loglik.sum())

In [7]:
prices = pd.read_hdf("data/equity-indices.h5", "sp500")
rets = 100 * prices.Close.pct_change().dropna()

mean = rets.mean()
var = rets.var()
starting_val = np.array([mean, var, 10])
full_std_t_loglik(starting_val, rets)

22751.097554147484

In [8]:
bounds = [(-10*np.abs(mean), 10*np.abs(mean)), 
          (var / 1000, 100 * var),
          (2.05, 100)]
opt = minimize(full_std_t_loglik, starting_val, args=(rets,), bounds=bounds,
               options={"disp": True})
print(f"The MLE is {opt.x} and the optimized (negative) LLF is {opt.fun}")

The MLE is [0.04885902 1.01909012 3.11731596] and the optimized (negative) LLF is 22034.20166109784


In [9]:
keys = ["sp500", "weekly_sp500", "monthly_sp500", "hsi", "weekly_hsi", "monthly_hsi"]
for key in keys:
    prices = pd.read_hdf("data/equity-indices.h5", key)
    rets = 100 * prices.Close.pct_change().dropna()
    
    mean = rets.mean()
    var = rets.var()
    starting_val = np.array([mean, var, 10])
   
    bounds = [(-10*np.abs(mean), 10*np.abs(mean)), 
              (var / 1000, 100 * var),
              (2.05, 100)]
    opt = minimize(full_std_t_loglik, starting_val, args=(rets,), bounds=bounds,
                   options={"disp": True})
    print(f"For {key}, the MLE is {opt.x} and the optimized (negative) LLF is {opt.fun}")


For sp500, the MLE is [0.04885902 1.01909012 3.11731596] and the optimized (negative) LLF is 22034.20166109784
For weekly_sp500, the MLE is [0.22792173 4.31086558 4.96479108] and the optimized (negative) LLF is 7646.416875193981
For monthly_sp500, the MLE is [ 0.83032412 16.92297255  6.9963708 ] and the optimized (negative) LLF is 2351.2108045225245

For hsi, the MLE is [0.04885902 1.01909012 3.11731596] and the optimized (negative) LLF is 22034.20166109784
For weekly_hsi, the MLE is [ 0.29490842 13.18069951  3.81006233] and the optimized (negative) LLF is 4459.7374461884465


For monthly_hsi, the MLE is [ 1.00200474 55.00150675  4.23240526] and the optimized (negative) LLF is 1316.7986085354805


### Exercise 22

Simulate a set of Bernoulli random variables $y_{i}$ where

$$p_{i}=\Phi\left(x_{i}\right)$$

where $X_{i}\sim N\left(0,1\right)$. (Note: $p_{i}$ is the probability of
success and $\Phi\left(\right)$ is the standard Normal CDF). Use this simulated data to
estimate the Probit model where $p_{i}=\Phi\left(\alpha_{0}+\alpha_{1}x_{i}\right)$ 
using maximum likelihood. 

In [10]:
from scipy import stats

nobs = 1000

x = rs.standard_normal(size=nobs)
p = stats.norm.cdf(x)
y = 1.0 * rs.random_sample(size=nobs) < p

In [11]:
def probit_llf(params, y, x):
    a0 = params[0]
    a1 = params[1]
    xhat = a0 + a1 * x
    phat = stats.norm.cdf(xhat)
    
    loglik = y * np.log(phat) + (1-y) * np.log(1-phat)
    return -(loglik.sum())

In [12]:
probit_llf([0, 1], y, x)

506.6527515404093

In [13]:
starting_val = np.array([0.0, 1.0])
opt = minimize(probit_llf, starting_val, args=(y, x))
print(f"The MLE is {opt.x}")

The MLE is [-0.04581815  0.99027212]


### Exercise 23

Estimate the asymptotic covariance of the estimated parameters in the previous.